### Bénin

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la page principale
url = "https://www.globenin.com/Benin/entreprises"
response = requests.get(url)

# Vérifier si la requête a réussi (code de statut 200 signifie succès)
if response.status_code == 200:
    # Analyser le contenu HTML de la page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Sélectionner la div avec la classe "CountryEntityList"
    div_country_entity_list = soup.find('div', class_='CountryEntityList')

    # Sélectionner tous les liens <a> à l'intérieur de la div et stocker leurs 'href' dans une liste
    liens = div_country_entity_list.find_all('a')
    liste_des_liens = [lien['href'] for lien in liens]

    # Préfixe de l'URL
    prefixe = "https://www.globenin.com/"

    # Liste pour stocker les liens complets
    liens_complets = []

    # Parcourir la liste des liens et les modifier pour les rendre complets
    for lien in liste_des_liens:
        lien_complet = prefixe + lien.replace("../", "")
        liens_complets.append(lien_complet)
    print(liens_complets)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
liste_des_tous_liens=[]
for lien in liens_complets:
    page = 1
    while True:
        url = f"{lien}/p{page}"
        try:
            response = requests.get(url)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Sélectionner toutes les divs avec la classe "listentity"
                divs_listentity = soup.find_all('div', class_='listentity')
               
                # Parcourez les divs pour extraire les liens
                for div_listentity in divs_listentity:
                    liens = div_listentity.find_all('a')
                    liste_des_liens = [lien['href'] for lien in liens]
                    # Ajoutez les liens à la liste des tous les liens
                    liste_des_tous_liens.extend(liste_des_liens)
                    for lien in liste_des_liens:
                        print(lien)

                next_page_link = soup.find('a', class_='pagination-next')
                if next_page_link:
                    page += 1
                else:
                    break
            else:
                print(f"Erreur lors de la récupération de la page {url}")
                break  # Sort de la boucle en cas d'erreur HTTP
        except Exception as e:
            print(f"Erreur lors de la récupération de la page {url}: {str(e)}")
            break  # Sort de la boucle en cas d'exception

# Assurez-vous d'ajuster les sélecteurs CSS et les actions spécifiques à votre cas d'utilisation

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Fonction pour extraire les informations à partir d'une URL
def extraire_informations(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Analyser le code HTML avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Initialiser une liste pour stocker les informations extraites de chaque div
        data = []

        # Sélectionner tous les div avec l'id "eHold"
        divs_eHold = soup.find_all('div', id='eHold')

        # Parcourir tous les divs eHold et extraire les informations
        for div_eHold in divs_eHold:
            # Extraire le nom de l'organisation (h1)
            nom_organisation_element = div_eHold.find('h1')
            nom_organisation = nom_organisation_element.text.strip() if nom_organisation_element else "Non disponible"

            # Extraire la catégorie (h2)
            categorie_element = div_eHold.find('h2')
            categorie = categorie_element.text.strip() if categorie_element else "Non disponible"

            # Extraire le statut juridique depuis la première balise p dans la div eHeadPres
            statut_juridique_element = div_eHold.select_one('div#eHeadPres p:nth-of-type(1)')
            statut_juridique = statut_juridique_element.text.strip() if statut_juridique_element else "Non disponible"

            # Extraire le secteur d'activité depuis la deuxième balise p dans la div eHeadPres
            secteur_activite_element = div_eHold.select_one('div#eHeadPres p:nth-of-type(2)')
            secteur_activite = secteur_activite_element.text.strip() if secteur_activite_element else "Non disponible"

            # Extraire tous les numéros de téléphone dans les span de classe "spvar" à l'intérieur de la div "eBody"
            numeros_telephone_elements = div_eHold.select('div#eBody span.spvar')
            numeros_telephone = [span.text.strip() for span in numeros_telephone_elements if '+' in span.text]
            
            # Extraire l'adresse email
            adresse_email_element = div_eHold.find('a', href=True, text=re.compile(r'[\w\.-]+@[\w\.-]+'))
            adresse_email = adresse_email_element['href'] if adresse_email_element else "Non disponible"

            # Extraire le site web
            site_web_element = div_eHold.find('a', href=True, text=re.compile(r'^https?://'))
            site_web = site_web_element['href'] if site_web_element else "Non disponible"
            
            contenu_spvar_texte = "Non disponible"
            # Parcourir tous les éléments <p> à l'intérieur de div_eHold
            for paragraphe_element in div_eHold.find_all('p'):
                strong_element = paragraphe_element.find('strong')
                # Vérifier si l'élément <strong> existe dans le paragraphe
                if strong_element:
                    spans_spvar = strong_element.find_all('span', class_='spvar')
                    # Extraire le contenu de tous les <span> avec la classe "spvar"
                    contenu_spvar = [span.text.strip() for span in spans_spvar]
                    contenu_spvar_texte = ', '.join(contenu_spvar)
                    # Sortir de la boucle une fois que le contenu a été trouvé
                    break
                    
            localisation = "Non disponible"
            ville = "Non disponible"
            # Parcourir tous les éléments <p> à l'intérieur de div_eHold
            for paragraphe_element in div_eHold.find_all('p'):
                splab_element = paragraphe_element.find('span', class_='splab')
                fas_city_element = paragraphe_element.find('span', class_='fas fa-city')
                fas_map_marker_element = paragraphe_element.find('span', class_='fas fa-map-marker-alt')

                # Vérifier si l'élément <span class="splab"> existe dans le paragraphe
                if splab_element:
                    spvar_element = paragraphe_element.find('span', class_='spvar')

                    # Vérifier si l'élément <span class="spvar"> existe
                    if spvar_element:
                        texte_spvar = spvar_element.text.strip()

                        # Vérifier si l'élément <span class="fas fa-map-marker-alt"> existe
                        if fas_map_marker_element:
                            localisation = texte_spvar
                        # Vérifier si l'élément <span class="fas fa-city"> existe
                        elif fas_city_element:
                            ville = texte_spvar
            # Extraire la référence
            reference_element = div_eHold.find('span', text='Réf.')
            reference = reference_element.find_next('span').text.strip() if reference_element else "Non disponible"

            # Ajouter les informations extraites dans la liste data
            data.append({
                "Nom de l'organisation": nom_organisation,
                "Catégorie": categorie,
                "Statut juridique": statut_juridique,
                "Secteur d'activité": secteur_activite,
                "Numéros de téléphone": ', '.join(numeros_telephone),
                "Adresse email": adresse_email,
                "Site web": site_web,
                "Référence": reference,
                "Nom de la personne": contenu_spvar_texte,
                "Localisation": localisation,  # Ajout de la localisation
                "Ville": ville  # Ajout de la ville
            })

        # Créer un DataFrame à partir de la liste de données
        df = pd.DataFrame(data)

        # Retourner le DataFrame
        return df

    else:
        print(f"Échec de la requête pour l'URL {url}. Code de statut : {response.status_code}")
        return None

    
# Créer une liste de DataFrames pour chaque URL
liste_de_dataframes = []

# Parcourir la liste des URL et extraire les informations pour chaque URL
for url in liste_des_tous_liens:
    df = extraire_informations(url)
    if df is not None:
        liste_de_dataframes.append(df)

# Concaténer tous les DataFrames en un seul DataFrame
dataframe_final = pd.concat(liste_de_dataframes, ignore_index=True)

# Afficher le DataFrame final
dataframe_final

In [ ]:
import pandas as pd

excel_file_path = "GL_BENIN.xlsx"

# Enregistrez le DataFrame dans le classeur Excel
dataframe_final.to_excel(excel_file_path, index=False)

# Confirmez l'enregistrement
print(f"Le DataFrame a été enregistré dans {excel_file_path}")

In [ ]:
import pandas as pd

chemin_fichier_excel = "GL_BENIN.xlsx"

df = pd.read_excel(chemin_fichier_excel)

# Importer pandas
import pandas as pd
# Remplacer les valeurs vides par 'Non disponible'
df.fillna('Non disponible', inplace=True)

# Nettoyer la colonne "Statut juridique"
df['Statut juridique'] = df['Statut juridique'].str.replace('Statut juridique:', '').str.strip()

# Nettoyer la colonne "Secteur d'activité"
df['Secteur d\'activité'] = df['Secteur d\'activité'].str.replace('Secteur d\'activité:', '').str.strip()

# Diviser la colonne "Numéros de téléphone" en une liste de numéros
df['Numéros de téléphone'] = df['Numéros de téléphone'].str.split(',')

# Créer des colonnes pour les numéros de téléphone, avec des valeurs par défaut "Non disponible"
df['Téléphone 1'] = 'Non disponible'
df['Téléphone 2'] = 'Non disponible'
df['Téléphone 3'] = 'Non disponible'

# Remplir les colonnes de téléphone avec les numéros disponibles
df['Téléphone 1'] = df['Numéros de téléphone'].apply(lambda x: x[0] if len(x) > 0 else 'Non disponible')
df['Téléphone 2'] = df['Numéros de téléphone'].apply(lambda x: x[1] if len(x) > 1 else 'Non disponible')
df['Téléphone 3'] = df['Numéros de téléphone'].apply(lambda x: x[2] if len(x) > 2 else 'Non disponible')

# Diviser la colonne "Nom de la personne" en deux colonnes distinctes (Nom et Poste)
df[['Nom de la personne', 'Poste']] = df['Nom de la personne'].str.split('/', expand=True)

# Nettoyer la colonne "Adresse email"
df['Adresse email'] = df['Adresse email'].str.replace('mailto:', '').str.strip()

df.drop(columns=['Numéros de téléphone'], inplace=True)
# Afficher le DataFrame nettoyé
df